In [ ]:
import os
import random
import numpy as np
import pandas as pd
from scipy.stats import norm

In [ ]:
random.seed(73)
np.random.seed(73)

In [ ]:
#-----------------------Auxiliary Functions----------------

def generate_dw_sample(dim, num_time_interval, num_sample, sqrt_delta_t):
    """Generate Brownian motion increments."""
    return np.random.randn(dim, num_time_interval, num_sample) * sqrt_delta_t

def generate_u_sample(dim, policy, num_sample):
    """Generate reference policies."""
    
    if POLICY == "even": 
        u_sample = np.ones((dim, num_sample)) * 1/dim
    
    elif POLICY == "random":
        u_sample = np.random.dirichlet(np.ones(dim), num_sample).T
        
    elif POLICY == "minimal": 
        u_sample = np.zeros((dim, num_sample))
    
    elif POLICY in {"best_2dim", "best_3dim"}:
        u_sample = np.zeros((dim, num_sample))
        u_sample[1,:] = np.ones(num_sample)
        
    elif POLICY == "best_3dim_var": 
        u_sample = np.zeros((dim, num_sample))
        u_sample[0,:] = np.ones(num_sample)
        
    elif POLICY == "weighted_split_main":
        u_sample = np.ones((DIM, NUM_SAMPLE)) * 0.037178922
        u_sample[0, :] = 0.159803922
        u_sample[1, :] = 0.159803922
        u_sample[2, :] = 0.159803922
    
    elif POLICY == "weighted_split_var1":
        alpha = 4/6
        base_weight = alpha * 0.025 + (1 - alpha) * 1/dim
        special_weight = alpha * 0.14 + (1 - alpha) * 1/dim
        u_sample = np.ones((dim, num_sample)) * base_weight
        u_sample[0,:] = special_weight
        u_sample[1,:] = special_weight
        u_sample[2,:] = special_weight
        u_sample[6,:] = special_weight
        u_sample[14,:] = special_weight
        
    elif POLICY == "weighted_split_var2":
        alpha = 3/6
        base_weight = alpha * 0.025 + (1 - alpha) * 1/dim
        special_weight = alpha * 0.14 + (1 - alpha) * 1/dim
        u_sample = np.ones((dim, num_sample)) * base_weight
        u_sample[0,:] = special_weight
        u_sample[1,:] = special_weight
        u_sample[2,:] = special_weight
        u_sample[6,:] = special_weight
        u_sample[14,:] = special_weight
        
    return u_sample

In [ ]:
# ------------------------------- Main Simulation Function --------------------------------

def sample(total_time, num_time_interval, dim, num_sample, policy, lambd, zeta, mu, theta):

    """Generate sample paths for the associated reference policy."""
    
    delta_t = total_time / num_time_interval
    sqrt_delta_t = np.sqrt(delta_t)
    num_five_min_interval = 204
    
    # Repeat values to match the required number of intervals
    lambd = np.repeat(lambd, num_time_interval // num_five_min_interval, axis=1)
    zeta = np.repeat(zeta, num_time_interval // num_five_min_interval, axis=1)
    
    sigma = np.sqrt(2 * lambd)   # Diffusion coefficient
    
    # Generate noise and control samples
    dw_sample = generate_dw_sample(dim, num_time_interval, num_sample, sqrt_delta_t)
    u_sample = generate_u_sample(dim, policy, num_sample)
    
    # Initialize sample paths
    x_sample = np.zeros((dim, num_time_interval + 1, num_sample)) 
    x_sample[:,0,:] = np.random.uniform(-10, 10, (dim, num_sample)) 

    for i in range(num_time_interval):
        
        sum_x = np.sum(x_sample[:, i, :], axis=0, keepdims=True)  # Shape (1, num_sample)
        mx = np.maximum(sum_x, 0.0)  # Shape (1, num_sample)
        
        zeta_i = zeta[:, i].reshape(-1, 1)
        sigma_i = sigma[:, i].reshape(-1, 1)
        
        x_sample[:, i + 1, :] = x_sample[:, i, :] + (zeta_i - mu * x_sample[:, i, :]) * delta_t \
                                + sigma_i * dw_sample[:, i, :] \
                                + (mx * ((mu - theta) * u_sample)) * delta_t
            
    return x_sample

In [ ]:
# ------------------------------- Fitting Functions --------------------------------
def fit_normal(x, dim, num_time_interval, policy, output_folder="output"):
    """Fit normal distributions to sample paths and save mean/std per interval."""

    #foldername = "/Users/ebrukasikaralar/100dim_main/records_" + policy + "_" + str(num_time_interval) + "_uniform_initialization"
    folder_path = os.path.join(output_folder, f"{policy}_{num_time_interval}_uniform_initialization")
    os.makedirs(folder_path, exist_ok=True)
    
    means = np.zeros((dim, num_time_interval + 1))
    stds = np.zeros((dim, num_time_interval + 1))
        
    for t in range(num_time_interval + 1):
        
        mean_std_file = os.path.join(folder_path, f"mean_std_interval{t}.csv")

        save_array = np.zeros((dim, 2))
            
        for cls in range(dim):
                        
            mean, std = norm.fit(x[cls, t])
            
            means[cls,t] = mean
            stds[cls,t] = std
                        
            save_array[cls,0] = mean
            save_array[cls,1] = std
            
            np.savetxt(mean_std_file, save_array, delimiter = ",", comments = "")
        
    return means, stds

In [ ]:
def saver(dim, num_time_interval, policy, output_folder="output"):
    """Load and save aggregated mean and standard deviation data."""
    
    folder_path = os.path.join(output_folder, f"{policy}_{num_time_interval}_uniform_initialization")
    file_name = "mean_std_interval"
    
    means = np.zeros((dim, num_time_interval + 1))
    stds = np.zeros((dim, num_time_interval + 1))
    
    for i in range(num_time_interval + 1):

        file = os.path.join(folder_path, f"{file_name}{i}.csv")
        data = pd.read_csv(file, header = None)

        mean = data[0]
        std = data[1]

        means[:, i] = mean
        stds[:, i] = std

    np.savetxt(os.path.join(folder_path, f"{policy}_{num_time_interval}_means.csv"), means, delimiter=",")
    np.savetxt(os.path.join(folder_path, f"{policy}_{num_time_interval}_stds.csv"), stds, delimiter=",")

In [ ]:
# ------------------------------- Sample simulation parameters --------------------------------
TOTAL_TIME = 17
NUM_TIME_INTERVAL = 1020
DIM = 17
NUM_SAMPLE = 5000
POLICY = "weighted_split_main"

# Load Data
MU = pd.read_csv("/Users/ebrukasikaralar/sample_path_generation/config_main/mu_hourly_17dim.csv", header=None).values.reshape((DIM, 1))
THETA = pd.read_csv("/Users/ebrukasikaralar/sample_path_generation/config_main/theta_hourly_17dim.csv", header=None).values.reshape((DIM, 1))
LAMBD = pd.read_csv("/Users/ebrukasikaralar/sample_path_generation/config_main/lambd_matrix_hourly_17dim.csv", header=None).values
ZETA = pd.read_csv("/Users/ebrukasikaralar/sample_path_generation/config_main/zeta_matrix_hourly_17dim.csv", header=None).values

In [ ]:
# ------------------------------- Run simulation --------------------------------
x_sample = sample(TOTAL_TIME, NUM_TIME_INTERVAL, DIM, NUM_SAMPLE, POLICY, LAMBD, ZETA, MU, THETA)

# Fit normal distributions and save results
fit_normal(x_sample, DIM, NUM_TIME_INTERVAL, POLICY)

# Save aggregated results
saver(DIM, NUM_TIME_INTERVAL, POLICY)